In [ ]:
drug = 'thiazide'
#drug = 'b-blocker'
#drug = 'ACEi'
#drug = 'ARB'
#drug = 'CCB'
#drug = 'ezetimibe'
#drug = 'fibrate'
#drug = 'statin'
#drug = 'insulin'
#drug = 'biguanide'
#drug = 'sulfonylurea'
#drug = 'thiazolidinedione'
#drug = 'antidiabetic_drugs_excl_biguanide_sulfonylurea'
#drug = 'bileacidsequestrant'
#drug = 'nicotinicacid'
#drug = 'omega3'
file_bnf    = f'bnf_{drug}.txt'
file_read_2 = f'read_2_{drug}.txt'
file_dmd    = f'dmd_{drug}.txt'
file_out    = f'{drug}.txt'

file_drug_name = f'drug_name_{drug}.txt' #thiazolidinedione

In [ ]:
import subprocess
output = subprocess.check_output(['wc', '-l', file_bnf, file_read_2, file_dmd])
print(output.decode('utf-8'))  # Decode bytes to string

In [ ]:
import pyspark
import dxpy
import dxdata
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

In [ ]:
import pandas as pd
import os

In [ ]:
dispensed_database_name = dxpy.find_one_data_object(classname="database", name="app*", folder="/", name_mode="glob", describe=True)["describe"]["name"]
dispensed_dataset_id = dxpy.find_one_data_object(typename="Dataset", name="app*.dataset", folder="/", name_mode="glob")["id"]

In [ ]:
spark.sql("USE " + dispensed_database_name)
spark.sql("SHOW TABLES").show(truncate=False)

In [ ]:
spark.sql("SELECT * FROM gp_scripts LIMIT 5").toPandas()

In [ ]:
result = spark.sql("SELECT * FROM gp_scripts LIMIT 0").toPandas()
result

In [ ]:
foo = pd.read_csv(file_bnf, sep='\t', header=None, dtype=object)
foo.shape[0]

In [ ]:
for k in foo.iloc[:,0]:
    k = k.rstrip()
    q = f"SELECT * FROM gp_scripts WHERE bnf_code LIKE '{k}' || '%'"
    df = spark.sql(q).toPandas()
    c = df.shape[0]
    print(q + f" ==> {c}")
    if c > 0:
        result = pd.concat([result, df])

In [ ]:
print(result.shape[0])
result = result.drop_duplicates()
print(result.shape[0])

In [ ]:
foo = pd.read_csv(file_read_2, sep='\t', header=None, dtype=object)
foo.shape[0]

In [ ]:
for k in foo.iloc[:,0]:
    k = k.rstrip()
    q = f"SELECT * FROM gp_scripts WHERE read_2 LIKE '{k}' || '%'"
    df = spark.sql(q).toPandas()
    c = df.shape[0]
    print(q + f" ==> {c}")
    if c > 0:
        result = pd.concat([result, df])

In [ ]:
print(result.shape[0])
result = result.drop_duplicates()
print(result.shape[0])

In [ ]:
foo = pd.read_csv(file_dmd, sep='\t', header=None, dtype=object)
foo.shape[0]

In [ ]:
for k in foo.iloc[:,0]:
    k = k.rstrip()
    q = f"SELECT * FROM gp_scripts WHERE dmd_code = '{k}'"
    df = spark.sql(q).toPandas()
    c = df.shape[0]
    print(q + f" ==> {c}")
    if c > 0:
        result = pd.concat([result, df])

In [ ]:
print(result.shape[0])
result = result.drop_duplicates()
print(result.shape[0])

In [ ]:
all_columns = result.columns.to_list()
result.sort_values(by=all_columns, inplace=True)

In [ ]:
result.head(5)

In [ ]:
result.to_csv(file_out, sep='\t', index=False)

In [ ]:
os.system(f'dx upload {file_out}')

# Extract earliest issue_date for each eid

- where clinical event or prescription date precedes participant date of birth it has been altered to 01/01/1901.
- Where the date matches participant date of birth it has been altered to 02/02/1902.
- Where the date follows participant date of birth but is in the year of their birth it has been altered to 03/03/1903.
- Where the date was in the future this has been changed to 07/07/2037 as these are likely to have been entered as a place-holder or other system default.

In [ ]:
result_df = spark.sql("""
    SELECT eid, MIN(issue_date) as earliest_issue_date
    FROM gp_scripts
    WHERE issue_date NOT IN ('1901-01-01', '1902-02-02', '1903-03-03', '2037-07-07')
    GROUP BY eid
""")

In [ ]:
result_pd = result_df.toPandas()

In [ ]:
result_pd

In [ ]:
result_pd.to_csv("earliest_issue_date.txt", sep="\t", index=False)